# Scrap appearance words

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd '/content/drive/MyDrive/Grad/1012/NLU1012_Project/WEAT Test'

/content/drive/.shortcut-targets-by-id/1vRARdXYACsDbBGzyCq5XDMyFJo8SzO1D/NLU1012_Project/WEAT Test


In [ ]:
from urllib.request import Request, urlopen
import ssl
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

# Scrap the appearance topic
url = "https://www.oxfordlearnersdictionaries.com/topic/appearance"
req = Request(url, headers=headers)
context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE
response = urlopen(req, context=context)
soup_apperance = BeautifulSoup(response, "html.parser")

appearance_words = {}
subtopics = ["attractiveness_t","body_shape_t","describing_hair_t","skin_t","styling_hair_t"]
for subtopic in subtopics:
    li_tags = soup_apperance.findAll("li", attrs={"data-"+subtopic: ["a1","a2","b1","b2","c1","c2"]})
    appearance_words[subtopic] = [li.get("data-hw") for li in li_tags]


# Scrap the body topic
url = "https://www.oxfordlearnersdictionaries.com/topic/body"
req = Request(url, headers=headers)
context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE
response = urlopen(req, context=context)
soup_body = BeautifulSoup(response, "html.parser")

body_words = {}
subtopics = ["face_t","hands_and_feet_t","mouth_and_teeth_t","parts_of_the_body_t"]
for subtopic in subtopics:
    li_tags = soup_body.findAll("li", attrs={"data-"+subtopic: ["a1","a2","b1","b2","c1","c2"]})
    body_words[subtopic] = [li.get("data-hw") for li in li_tags]


# Scrap the clothes topic
url = "https://www.oxfordlearnersdictionaries.com/topic/clothes-and-fashion"
req = Request(url, headers=headers)
context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE
response = urlopen(req, context=context)
soup_clothes = BeautifulSoup(response, "html.parser")

clothes_words = {}
subtopics = ["accessories_t","describing_clothes_t","footwear_t","hats_t","items_of_clothing_t","jewellery_t","parts_of_clothing_t"]
for subtopic in subtopics:
    li_tags = soup_clothes.findAll("li", attrs={"data-"+subtopic: ["a1","a2","b1","b2","c1","c2"]})
    clothes_words[subtopic] = [li.get("data-hw") for li in li_tags]



# Count the scraped word in each sub-topic
print("Scrapping results:")
for k,v in appearance_words.items():
    print(k, len(v))

print("-"*20)

for k,v in body_words.items():
    print(k, len(v))

print("-"*20)

for k,v in clothes_words.items():
    print(k, len(v))


# Put together all target words
appearnace_target = []
for i in appearance_words.values():
    appearnace_target.extend(i)

body_target = []
for i in body_words.values():
    body_target.extend(i)

clothes_target = []
for i in clothes_words.values():
    clothes_target.extend(i)

target_word_complete = list(set(appearnace_target+body_target+clothes_target))

Scrapping results:
attractiveness_t 67
body_shape_t 53
describing_hair_t 134
skin_t 42
styling_hair_t 77
--------------------
face_t 41
hands_and_feet_t 29
mouth_and_teeth_t 30
parts_of_the_body_t 63
--------------------
accessories_t 32
describing_clothes_t 134
footwear_t 63
hats_t 56
items_of_clothing_t 149
jewellery_t 54
parts_of_clothing_t 30


In [ ]:
len(target_word_complete)

1004

In [ ]:
file = open("target_word_complete.txt","w")

for item in target_word_complete:
    file.write(item+"\n")

file.close()

# WEAT Test

In [ ]:
import numpy as np
from embeddings import Embeddings
from test_analogies import cosine_sim, get_closest_words, load_analogies, run_analogy_test

from scipy.stats import permutation_test

In [ ]:
# Load continue trained embeddings
from gensim.models import Word2Vec

# Load fine-tuned embeddings
je_model = Word2Vec.load("word2vec.model")
ge_model = Word2Vec.load("great_expectation.model")
lo_model = Word2Vec.load("lolita.model")
pp_model = Word2Vec.load("pp.model")

# Load glove embeddings
embeddings = Embeddings.from_file("glove.6B.50d.txt")

In [ ]:
def target_filter(embeddings, target_word_complete):
    target_word = []
    for w in target_word_complete:
        if w in embeddings:
            target_word.append(w)
    print(f'# of target words: {len(target_word)}')
    target = embeddings[target_word]
    return target

In [ ]:
def attribute(word_list, embedding):
    words_lower = [w.lower() for w in word_list]
    attribute_vectors = embedding[words_lower]
    return attribute_vectors

In [ ]:
def statistic(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

# def test(f_attribute, m_attribute, target):
#     res = permutation_test((cosine_sim(f_attribute,target).flatten(), cosine_sim(m_attribute,target).flatten()), 
#                        statistic, 
#                        n_resamples=5000,
#                        vectorized=True, 
#                        alternative="greater")
#                     #    random_state=42)
#     print(res)

## Baseline

In [ ]:
f_typical = ["woman","female","girl","she","her","herself","wife","mother","miss","lady"]
m_typical = ["man","male","boy","he","him","himself","husband","father","mr","sir"]

f_attribute = attribute(f_typical, embeddings)
m_attribute = attribute(m_typical, embeddings)
target = target_filter(embeddings, target_word_complete)
res = permutation_test((cosine_sim(f_attribute,target).flatten(), cosine_sim(m_attribute,target).flatten()), 
                       statistic, 
                       n_resamples=10000,
                       vectorized=True, 
                       alternative="two-sided")
print(res)

# of target words: 828
PermutationTestResult(statistic=0.06378414030269516, pvalue=0.00019998000199980003, null_distribution=array([-4.59942773e-03,  7.08342620e-05,  5.09863678e-04, ...,
        4.99811732e-03, -4.11600123e-04, -1.49358266e-03]))


## Jane Eyre Results

In [ ]:
# Construct the attribute lists
je_f = ["Jane",
"Eyre",
"Janet",
"Sarah",
"Reed",
"Gibson",
"Eliza",
"Georgiana",
"Bessie",
"Lee",
"Martha",
"Abbot",
"Maria",
"Temple",
"Scatcherd",
"Helen",
"Burns",
"Alice",
"Fairfax",
"Varens",
"Grace",
"Poole",
"Leah",
"Bertha",
"Antoinetta",
"Mary",
"Diana",
"Rosamond",
"Sophie",
"Eshton",
"Miller",
"Hannah"]
je_m = ["John",
"Lloyd",
"Brocklehurst",
"Edward",
"Rochester",
"Blanche",
"Ingram",
"Richard",
"Mason",
"Robert",
"Leaven",
"John",
"Rivers",
"Briggs",
"Oliver"]


f_attribute = attribute(je_f+f_typical, je_model.wv)
m_attribute = attribute(je_m+m_typical, je_model.wv)
target = filter(je_model.wv, target_word_complete)
res = permutation_test((cosine_sim(f_attribute,target).flatten(), cosine_sim(m_attribute,target).flatten()), 
                       statistic, 
                       n_resamples=10000,
                       vectorized=True, 
                       alternative="two-sided")
print(res)

# of target words: 302
PermutationTestResult(statistic=0.03248871640416809, pvalue=0.00019998000199980003, null_distribution=array([ 0.0064793 , -0.00248976,  0.00023541, ..., -0.00147717,
       -0.00097195,  0.00314186]))


## Great Expectation Results

In [ ]:
# Construct the attribute lists
ge_f = ["Havisham", "Estella","Biddy","Skiffins","Sarah","Clara","Jane","Camilla","Mum", "Hubble","Flopson","Jane","Coiler", "Whimple"]
ge_m = ["Joe", "Herbert", "Wemmick","Jaggers","Pip","Pumblechook","Wopsle","Provis","Drummle","Orlick","Handel","Joseph","Jack","Trabb","Pumblechook","Gargery","Matthew","Magwitch","Mike","Avenger","Richard","Tom","John","Waldengarver","Compeyson","Spider","Bill","William", "Startop","Brandley","Arthur","Raymond", "Barley","Barnard","Campbell"]

f_attribute = attribute(ge_f+f_typical, ge_model.wv)
m_attribute = attribute(ge_m+m_typical, ge_model.wv)
target = filter(ge_model.wv, target_word_complete)
res = permutation_test((cosine_sim(f_attribute,target).flatten(), cosine_sim(m_attribute,target).flatten()), 
                       statistic, 
                       n_resamples=5000,
                       vectorized=True, 
                       alternative="greater",
                       random_state=100)
print(res)

# of target words: 267
PermutationTestResult(statistic=0.03277794583010285, pvalue=0.0001999600079984003, null_distribution=array([-0.00155281,  0.00221242, -0.00482424, ..., -0.00146112,
        0.00404935,  0.00337777]))


## Lolita

In [ ]:
lo_f = ["Lolita","Haze","Dolores","Lo","Charlotte","Annabel","Leigh","Valeria","Jean","Dick","Schiller","Rita","Monique","Mona","Louise","Shirley","Holmes","Pratt","Barbara","Vivian","Darkbloom","Mary","Lore","Avis","Dolly","Holigan"]
lo_m = ["Humbert","Clare","Quilty","John","Ivor","Ray","Charlie","Frederick","Beale","Gaston","Windmuller","McCoo","Bill","Leslie","Phyllis","Frank","Tony"]

f_attribute = attribute(lo_f+f_typical, lo_model.wv)
m_attribute = attribute(lo_m+m_typical, lo_model.wv)
target = filter(lo_model.wv, target_word_complete)
res = permutation_test((cosine_sim(f_attribute,target).flatten(), cosine_sim(m_attribute,target).flatten()), 
                       statistic, 
                       n_resamples=5000,
                       vectorized=True, 
                       alternative="greater",
                       random_state=100)
print(res)

# of target words: 432
PermutationTestResult(statistic=0.020038835917551956, pvalue=0.0001999600079984003, null_distribution=array([ 0.0007749 , -0.00488987, -0.0002773 , ..., -0.00137677,
       -0.0019925 , -0.00746694]))


## Pride and prejudice

In [ ]:
pp_f = ["Elizabeth",
"Jane",
"Bingley",
"Lizzy",
"Kitty",
"Lydia",
"Mary",
"Catherine",
"Bennet",
"Jenkinson",
"Maria",
"Bourgh",
"Reynolds",
"Caroline",
"King",
"Charlotte",
"Louisa",
"Hill",
"Eliza"]
pp_m = ["Darcy",
"Collins",
"Wickham",
"Fitzwilliam",
"Colonel",
"William",
"Carter",
"Denny",
"Charles",
"Jones"]

f_attribute = attribute(pp_f+f_typical, pp_model.wv)
m_attribute = attribute(pp_m+m_typical, pp_model.wv)
target = filter(pp_model.wv, target_word_complete)
res = permutation_test((cosine_sim(f_attribute,target).flatten(), cosine_sim(m_attribute,target).flatten()), 
                       statistic, 
                       n_resamples=5000,
                       vectorized=True, 
                       alternative="greater",
                       random_state=100)
print(res)

# of target words: 133
PermutationTestResult(statistic=-0.0037854372273905257, pvalue=0.7870425914817036, null_distribution=array([-0.00305704,  0.00335078, -0.00274429, ..., -0.00176769,
        0.00053072,  0.0007094 ]))


# Error Analysis

In [ ]:
count = 0
for i in je_model.wv.most_similar('rochester',topn=1000):
    if i[0] in target_word_complete:
        print(i[0])
        count += 1

count

barber
winning
temple
becoming
back
part
straight
belle
appearance
ball
crown
long
top
bridge
match


15